In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
from kornia import lab_to_rgb
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision.models as models
from torchvision import datasets, transforms
# For utilities
import os, shutil, time
from torch.autograd import Variable

from torchmetrics import PSNR
from torchmetrics import SSIM
from torchmetrics import FID

In [2]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()

In [3]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()

    ## First half: ResNet
    resnet = models.resnet18(num_classes=313) 
    # Change first conv layer to accept single-channel (grayscale) input
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Second half: Upsampling
    self.upsample = nn.Sequential(
      nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0),
      nn.Softmax(dim=1),
      nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1),
      nn.Upsample(scale_factor=4))
              
  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

In [4]:
model = ColorizationNet()
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

In [5]:
class GrayscaleImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is not None:
      img_original = self.transform(img)
      img_original = np.asarray(img_original)
      img_lab = rgb2lab(img_original)
      img_lab = (img_lab + 128) / 255
      img_ab = img_lab[:, :, 1:3]
      img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
      img_original = rgb2gray(img_original)
      img_original = torch.from_numpy(img_original).unsqueeze(0).float()
    if self.target_transform is not None:
      target = self.target_transform(target)
    return img_original, img_ab, target

In [6]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [7]:
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip()])
train_imagefolder = GrayscaleImageFolder('images/train', train_transforms)
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=64, shuffle=True)

# Validation 
val_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])
val_imagefolder = GrayscaleImageFolder('images/val' , val_transforms)
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=64, shuffle=False)

In [8]:
psnrMetric = PSNR()
if use_gpu: psnrMetric = psnrMetric.cuda()
def calculatePSNR(output_ab, input_ab):
  if use_gpu: output_ab, input_ab =  output_ab.cuda(), input_ab.cuda()
  result = psnrMetric(output_ab, input_ab)
  psnrMetric.reset()
  return result.cuda().detach()

In [9]:
ssimMetric = SSIM()
if use_gpu: ssimMetric = ssimMetric.cuda()
def calculateSSIM(output_ab, input_ab):
  if use_gpu: output_ab, input_ab =  output_ab.cuda(), input_ab.cuda()
  result = ssimMetric(output_ab, input_ab)
  ssimMetric.reset()
  return result.cuda().detach()

C:\Users\VargasKiller\anaconda3\envs\ImageColorizationEnv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `SSIM` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [10]:
def prueba(gray, color):
    color_image = torch.cat((gray, color), 1).numpy() # combine channels
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
    color_image = lab_to_rgb(torch.tensor(color_image))
    return torch.from_numpy(color_image.numpy()).to(torch.uint8)

In [11]:
fidMetric = FID(feature=64)
if use_gpu: fidMetric = fidMetric.cuda()
def calculateFID(input_gray, output_ab, input_ab):
    input_cielab = prueba(input_gray, input_ab)
    output_cielab = prueba(input_gray, output_ab)  
 
    fidMetric.update(input_cielab, real=True)  
    fidMetric.update(output_cielab, real=False)  
    del(input_cielab)
    del(output_cielab)
    result = fidMetric.compute()
    fidMetric.reset()
    return result.cuda().detach()

C:\Users\VargasKiller\anaconda3\envs\ImageColorizationEnv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `FID` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [12]:
def calculate_metrics(output_ab, input_ab, gray):
    return calculatePSNR(output_ab, input_ab), calculateSSIM(output_ab, input_ab), 0
    #return calculatePSNR(output_ab, input_ab), calculateSSIM(output_ab, input_ab), calculateFID(gray,output_ab,input_ab)

In [13]:
def validate(val_loader, model, criterion, save_images, epoch):
  model.eval()

  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  PSNR_ACCUMULATE, SSIM_ACCUMULATE, FID_ACCUMULATE = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  already_saved_images = False
  for i, (input_gray, input_ab, target) in enumerate(val_loader):
    data_time.update(time.time() - end)

    # Use GPU
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()
    # Run model and record loss
    output_ab = model(input_gray) # throw away class predictions
    PSNR, SSIM, FID = calculate_metrics(output_ab, input_ab, input_gray)
    #print("metrics ", PSNR, SSIM, PID)
    loss = criterion(output_ab, input_ab)
    losses.update(loss.item(), input_gray.size(0))
    PSNR_ACCUMULATE.update(PSNR.item(), input_gray.size(0))
    SSIM_ACCUMULATE.update(SSIM.item(), input_gray.size(0))
    FID_ACCUMULATE.update(FID, input_gray.size(0))
    del(PSNR)
    del(SSIM)
    del(FID)
    # Save images to file
    if save_images and not already_saved_images:
      already_saved_images = True
      for j in range(min(len(output_ab), 10)): # save at most 5 images
        save_path = {'grayscale': 'outputs/gray/', 'colorized': 'outputs/color/'}
        save_name = 'img-{}-epoch-{}.jpg'.format(i * val_loader.batch_size + j, epoch)
        to_rgb(input_gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    # Record time to do forward passes and save images
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to both value and validation
    if i % 25 == 0:
      print('Validate: [{0}/{1}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
             i, len(val_loader), batch_time=batch_time, loss=losses))

  print('Finished validation.')
  return losses.avg, PSNR_ACCUMULATE.avg, SSIM_ACCUMULATE.avg, FID_ACCUMULATE.avg

In [14]:

# Import writer class from csv module
from csv import writer
  
def saveTrainingMetrics(trainData, file):  
    # Open our existing CSV file in append mode
    # Create a file object for this file
    with open(file, 'a') as f_object:

        # Pass this file object to csv.writer()
        # and get a writer object
        writer_object = writer(f_object)

        # Pass the list as an argument into
        # the writerow()
        writer_object.writerow(trainData)

        #Close the file object
        f_object.close()

In [15]:
def train(train_loader, model, criterion, optimizer, epoch):
  print('Starting training epoch {}'.format(epoch))
  model.train()
  
  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  for i, (input_gray, input_ab, target) in enumerate(train_loader):
    
    # Use GPU if available
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Record time to load data (above)
    data_time.update(time.time() - end)

    # Run forward pass
    output_ab = model(input_gray)
    loss = criterion(output_ab, input_ab)
    losses.update(loss.item(), input_gray.size(0))

    # Compute gradient and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record time to do forward and backward passes
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to value, not validation
    if i % 25 == 0:
      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
              epoch, i, len(train_loader), batch_time=batch_time,
             data_time=data_time, loss=losses)) 

  print('Finished training epoch {}'.format(epoch))

In [16]:
# Move model and loss function to GPU
if use_gpu:
    print("GPU Active")
    criterion = criterion.cuda()
    model = model.cuda()
    
    

GPU Active


In [17]:
# Make folders and set parameters
os.makedirs('outputs/color', exist_ok=True)
os.makedirs('outputs/gray', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 100

In [ ]:

# Train model
print(epochs)
tolerance = 20
earlyStopCounter = 0
for epoch in range(epochs):
  train(train_loader, model, criterion, optimizer, epoch)
  with torch.no_grad():
    losses = validate(val_loader, model, criterion, save_images, epoch)
    trainData = [epoch,losses[0]]
    metricsData = [epoch,losses[1:]]
    print(epoch, losses[0])
    saveTrainingMetrics(trainData,'T0104_DeepPaint_SmootL1.csv')
    saveTrainingMetrics(metricsData,'T0104_DeepPaint_SmootL1_Metrics.csv')
  # Save checkpoint and replace old best model if current model is better
  if losses[0] < best_losses:
    earlyStopCounter = 0
    best_losses = losses[0]
    torch.save(model.state_dict(), 'checkpoints/model-epoch-{}-losses-{:.8f}.pth'.format(epoch+1,losses[0]))
  else:
    earlyStopCounter += 1
    if earlyStopCounter > tolerance:
        print("Early Stop at",earlyStopCounter)
        break
    

100
Starting training epoch 0


C:\Users\VargasKiller\anaconda3\envs\ImageColorizationEnv\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/157]	Time 2.363 (2.363)	Data 1.157 (1.157)	Loss 0.1349 (0.1349)	
Epoch: [0][25/157]	Time 1.186 (1.226)	Data 1.141 (1.135)	Loss 0.0018 (0.0410)	
Epoch: [0][50/157]	Time 1.178 (1.203)	Data 1.133 (1.135)	Loss 0.0011 (0.0221)	
Epoch: [0][75/157]	Time 1.198 (1.202)	Data 1.154 (1.141)	Loss 0.0018 (0.0153)	
Epoch: [0][100/157]	Time 1.191 (1.199)	Data 1.146 (1.142)	Loss 0.0017 (0.0119)	
Epoch: [0][125/157]	Time 1.189 (1.196)	Data 1.143 (1.141)	Loss 0.0017 (0.0099)	
Epoch: [0][150/157]	Time 1.204 (1.197)	Data 1.159 (1.143)	Loss 0.0022 (0.0085)	
Finished training epoch 0
Validate: [0/16]	Time 1.915 (1.915)	Loss 0.0016 (0.0016)	
Finished validation.
0 0.0016117938393726944
Starting training epoch 1
Epoch: [1][0/157]	Time 1.217 (1.217)	Data 1.171 (1.171)	Loss 0.0017 (0.0017)	
Epoch: [1][25/157]	Time 1.154 (1.178)	Data 1.109 (1.133)	Loss 0.0015 (0.0016)	
Epoch: [1][50/157]	Time 1.194 (1.171)	Data 1.147 (1.126)	Loss 0.0015 (0.0016)	
Epoch: [1][75/157]	Time 1.221 (1.179)	Data 1.173 (1.13

In [ ]:
# Show images 
import matplotlib.image as mpimg
image_pairs = [('outputs/color/img-1-epoch-1.jpg', 'outputs/gray/img-1-epoch-0.jpg'),
               ('outputs/color/img-2-epoch-1.jpg', 'outputs/gray/img-2-epoch-0.jpg')]
for c, g in image_pairs:
  color = mpimg.imread(c)
  gray  = mpimg.imread(g)
  f, axarr = plt.subplots(1, 2)
  f.set_size_inches(15, 15)
  axarr[0].imshow(gray, cmap='gray')
  axarr[1].imshow(color)
  axarr[0].axis('off'), axarr[1].axis('off')
  plt.show()

In [ ]:
imgs_dist1 = torch.randint(0, 200, (100, 3, 299, 299), dtype=torch.uint8)  
imgs_dist2 = torch.randint(100, 255, (100, 3, 299, 299), dtype=torch.uint8)
print(imgs_dist1.shape, imgs_dist2.shape)